In [ ]:
!pip3 install git+https://github.com/Centre-automatique-et-systemes/lena.git gwpy &> /dev/null
!pip3 install git+https://github.com/aliutkus/torchinterp1d.git gwpy &> /dev/null

In [3]:
import sys ; sys.path.append('../')
import torch.optim as optim
import torch
import seaborn as sb
import pytorch_lightning as pl
import numpy as np

from sklearn.model_selection import train_test_split
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from learn_KKL.luenberger_observer import LuenbergerObserver
from learn_KKL.system import HO_unknown_freq
from learn_KKL.learner import Learner

sb.set_style('whitegrid')

In [4]:
# Set up the system
system = HO_unknown_freq()

In [5]:
# Instantiate the observer
D = torch.diag_embed(torch.tensor([-1.5, -2.0, -3.0, -4.0]))
#observer = LuenbergerObserver(dim_x=3, dim_y=1, method='Supervised', wc=0.2,
#                              recon_lambda=0.8)
observer = LuenbergerObserver(dim_x=3, dim_y=1, method='Supervised', wc=0.2, D=D,
                              recon_lambda=0.8)
observer.set_dynamics(system)
# Generate (x_i, z_i) data by running system backward, then system + observer
# forward in time
data = observer.generate_data_svl(np.array([[-1, 1.], [-1., 1.], [-1.,1.]]), 7200,
                                  method='uniform')
print(data.shape)
data, val_data = train_test_split(data, test_size=0.3, shuffle=True)

# Train the forward transformation using pytorch-lightning and the learner class
# Options for training
trainer_options={'max_epochs': 15}
optimizer_options = {'weight_decay': 1e-6}
scheduler_options = {'mode': 'min', 'factor': 0.1, 'patience': 3,
                     'threshold': 1e-4, 'verbose': True}
stopper = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss', min_delta=5e-4, patience=3, verbose=False, mode='min')
# Instantiate learner
learner_T = Learner(observer=observer, system=system, training_data=data,
                    validation_data=val_data, method='T', batch_size=10,
                    lr=1e-3, optimizer=optim.Adam,
                    optimizer_options=optimizer_options,
                    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                    scheduler_options=scheduler_options)
# Define logger and checkpointing
logger = TensorBoardLogger(save_dir=learner_T.results_folder + '/tb_logs')
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
trainer = pl.Trainer(
    callbacks=[stopper, checkpoint_callback], **trainer_options, logger=logger,
    log_every_n_steps=1, check_val_every_n_epoch=3)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
torch.Size([4704, 7])
Results saved in in c:\Users\Pauline\Documents\Pro\Recherche\20_LearningTransfoLuenberger\CodeLukas\learn_observe_KKL\jupyter_notebooks\runs\HO_unknown_freq\Supervised/T\exp_2


In [6]:
# To see logger in tensorboard, copy the following output name_of_folder
print(f'Logs stored in {learner_T.results_folder}/tb_logs')
# which should be similar to jupyter_notebooks/runs/method/exp_0/tb_logs/
# Then type this in terminal:
# tensorboard --logdir=name_of_folder --port=8080

# Train and save results
trainer.fit(learner_T)
learner_T.save_results(limits=np.array([[-1, 1.], [-1., 1.], [-1., 1.]]),
                       nb_trajs=10, tsim=(0, 10), dt=1e-4,
                       checkpoint_path=checkpoint_callback.best_model_path)


  | Name  | Type               | Params
---------------------------------------------
0 | model | LuenbergerObserver | 26.3 K
---------------------------------------------
26.3 K    Trainable params
0         Non-trainable params
26.3 K    Total params
0.105     Total estimated model params size (MB)
Logs stored in c:\Users\Pauline\Documents\Pro\Recherche\20_LearningTransfoLuenberger\CodeLukas\learn_observe_KKL\jupyter_notebooks\runs\HO_unknown_freq\Supervised/T\exp_2/tb_logs
Epoch 2:  70%|██████▉   | 330/472 [00:02<00:01, 140.58it/s, loss=4.63e-05, v_num=0, train_loss=2.96e-5]
Validating: 0it [00:00, ?it/s]
Epoch 5:  70%|██████▉   | 330/472 [00:02<00:01, 132.30it/s, loss=1.23e-05, v_num=0, train_loss=1.49e-5, val_loss_step=8.32e-6, val_loss_epoch=2.01e-5]
Validating: 0it [00:00, ?it/s]
Epoch 8:  70%|██████▉   | 330/472 [00:02<00:01, 141.55it/s, loss=1.49e-05, v_num=0, train_loss=1.8e-6, val_loss_step=3.23e-5, val_loss_epoch=1.24e-5] 
Validating: 0it [00:00, ?it/s]
Epoch 11:  70%|████

0

In [17]:
# Train the inverse transformation using pytorch-lightning and the learner class
# Options for training
trainer_options={'max_epochs': 20}
optimizer_options = {'weight_decay': 1e-8}
scheduler_options = {'mode': 'min', 'factor': 0.1, 'patience': 3,
                     'threshold': 1e-4, 'verbose': True}
stopper = pl.callbacks.early_stopping.EarlyStopping(
    monitor='val_loss', min_delta=5e-4, patience=3, verbose=False, mode='min')
# Instantiate learner
learner_T_star = Learner(observer=observer, system=system, training_data=data,
                         validation_data=val_data, method='T_star',
                         batch_size=10, lr=5e-4, optimizer=optim.Adam,
                         optimizer_options=optimizer_options,
                         scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                         scheduler_options=scheduler_options)
# Define logger and checkpointing
logger = TensorBoardLogger(save_dir=learner_T_star.results_folder + '/tb_logs')
checkpoint_callback = ModelCheckpoint(monitor='val_loss')
trainer = pl.Trainer(
    callbacks=[stopper, checkpoint_callback], **trainer_options, logger=logger,
    log_every_n_steps=1, check_val_every_n_epoch=3)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Results saved in in c:\Users\Pauline\Documents\Pro\Recherche\20_LearningTransfoLuenberger\CodeLukas\learn_observe_KKL\jupyter_notebooks\runs\HO_unknown_freq\Supervised/T_star\exp_3


In [18]:
# To see logger in tensorboard, copy the following output name_of_folder
print(f'Logs stored in {learner_T_star.results_folder}/tb_logs')
# which should be similar to jupyter_notebooks/runs/method/exp_0/tb_logs/
# Then type this in terminal:
# tensorboard --logdir=name_of_folder --port=8080

# Train and save results
trainer.fit(learner_T_star)
learner_T_star.save_results(limits=np.array([[-1, 1.], [-1., 1.], [-1.,1.]]),
                            nb_trajs=10, tsim=(0, 10), dt=1e-4,
                            checkpoint_path=checkpoint_callback.best_model_path)


  | Name  | Type               | Params
---------------------------------------------
0 | model | LuenbergerObserver | 26.3 K
---------------------------------------------
26.3 K    Trainable params
0         Non-trainable params
26.3 K    Total params
0.105     Total estimated model params size (MB)
Logs stored in c:\Users\Pauline\Documents\Pro\Recherche\20_LearningTransfoLuenberger\CodeLukas\learn_observe_KKL\jupyter_notebooks\runs\HO_unknown_freq\Supervised/T_star\exp_3/tb_logs
Epoch 2:  70%|██████▉   | 330/472 [00:02<00:01, 118.77it/s, loss=0.36, v_num=0, train_loss=0.371] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  70%|██████▉   | 330/472 [00:02<00:01, 121.08it/s, loss=0.372, v_num=0, train_loss=0.284, val_loss_step=0.346, val_loss_epoch=0.367]
Validating: 0it [00:00, ?it/s]
Epoch 8:  70%|██████▉   | 330/472 [00:02<00:01, 118.85it/s, loss=0.375, v_num=0, train_loss=0.536, val_loss_step=0.591, val_loss_epoch=0.367]
Validating: 0it [00:00, ?it/s]
Epoch 8: 100%|██████████| 472/472 [00

In [9]:

# We compare the learned T to the theoretical T
# T(x) = C (A(x_3)-lambda I)^{-1} [x_1;x_2]
# with C = [1,0] and A(x_3) = [0, 1; -x_3, 0]
# (this is for a diagonal D and F = ones)

N = learner_T.validation_data.size()[0]
T_theo = torch.zeros(N,4)
lambda_D = torch.diag(D)
A = torch.tensor([[0.,1.],[-learner_T.validation_data[17,2],0.]])
C = torch.tensor([[1.,0.]])
x12 = learner_T.validation_data[17,:2]
for ind in range(N):
    A = torch.tensor([[0.,1.],[-learner_T.validation_data[ind,2],0.]])
    x12 = learner_T.validation_data[ind,:2]
    #print(ind)
    for ind_eig in range(4):
        #print(torch.linalg.eig(A-lambda_D[ind_eig]*torch.eye(2)))
        T_theo[ind,ind_eig] =  C@torch.linalg.inv(A-lambda_D[ind_eig]*torch.eye(2))@x12


error = abs(T_theo-learner_T.validation_data[:,3:])
print(error.max())

tensor(0.0716)
